# Selected socio-economic determinants of social trust

Author: Mateusz Kasprowicz
Date: January 2024

## Load libraries

In [22]:
import random
import os

import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

## Constants

In [19]:
random_state = 0

np.random.seed(random_state)
os.environ["PYTHONHASHSEED"] = str(random_state)
random.seed(random_state)

In [20]:
sklearn.set_config(transform_output="pandas")

## Load data

In [3]:
columns_used = ["cntry", 
                "agea", 
                "eduyrs", 
                "gndr", 
                "domicil", # A big city, suburbs, town or small city, country village, etc.
                "hinctnta", # Household's total net income, all sources
                "uemp3m", # Ever unemployed and seeking work for a period more than three months
                "ppltrst", # Most people can be trusted or you can't be too careful
                ]

In [4]:
data = pd.read_stata(r"../data/ESS10SC_STATA/ESS10SC.dta", columns=columns_used)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22074 entries, 0 to 22073
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   cntry     22074 non-null  object  
 1   agea      21028 non-null  category
 2   eduyrs    20074 non-null  category
 3   gndr      21439 non-null  category
 4   domicil   21291 non-null  category
 5   hinctnta  17232 non-null  category
 6   uemp3m    20676 non-null  category
 7   ppltrst   21921 non-null  category
dtypes: category(7), object(1)
memory usage: 328.7+ KB


In [6]:
data[["agea", "eduyrs"]] = data[["agea", "eduyrs"]].astype(pd.Int64Dtype())

In [7]:
data.sample(5)

,cntry,agea,eduyrs,gndr,domicil,hinctnta,uemp3m,ppltrst
6676,DE,77,<NA>,Female,NaN,NaN,NaN,2
14824,IL,42,16,Male,Suburbs or outskirts of big city,H - 10th decile,Yes,7
18791,RS,61,12,Female,Town or small city,C - 3rd decile,Yes,You can't be too careful
11088,DE,51,13,Female,Town or small city,S - 6th decile,No,6
8420,DE,33,15,Male,Country village,R - 2nd decile,No,3


## Overall EDA

In [ ]:
profile = ProfileReport(data, title="Profiling Report")
profile.to_file("../data/EDA_full.html")

## Modeling

### Model for Poland

#### Preprocess data

In [9]:
data_pl = data[data.cntry == "PL"].drop(columns=["cntry"])

In [14]:
data_pl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2065 entries, 16217 to 18281
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   agea      1972 non-null   Int64   
 1   eduyrs    1944 non-null   Int64   
 2   gndr      2065 non-null   category
 3   domicil   1989 non-null   category
 4   hinctnta  1579 non-null   category
 5   uemp3m    1940 non-null   category
 6   ppltrst   2041 non-null   category
dtypes: Int64(2), category(5)
memory usage: 63.7 KB


In [15]:
data_pl.isna().sum()

agea         93
eduyrs      121
gndr          0
domicil      76
hinctnta    486
uemp3m      125
ppltrst      24
dtype: int64

In [ ]:
# ProfileReport(data_pl, title="Profiling Report for Poland").to_file("../data/EDA_PL.html")

Number of observations with at least one NaN value

In [34]:
data_pl.loc[data_pl.isna().any(axis=1)].shape[0]

600

In [30]:
X_pl = data_pl.dropna().drop(columns=["ppltrst"])
y_pl = data_pl.dropna()["ppltrst"]

In [41]:
X_pl[["agea", "eduyrs"]] = X_pl[["agea", "eduyrs"]].astype(int)

In [51]:
encoder = make_column_transformer((OneHotEncoder(drop="first", sparse=False), make_column_selector(dtype_exclude=np.number)), 
                                  remainder="passthrough",
                                  n_jobs=-1, 
                                  verbose=True, 
                                  verbose_feature_names_out=False,
                                  )

In [52]:
X_pl_transformed = encoder.fit_transform(X_pl)

#### Build a model

In [ ]:
# ordinal logistic regression: https://www.statsmodels.org/stable/examples/notebooks/generated/ordinal_regression.html
# https://www.statsmodels.org/dev/generated/statsmodels.miscmodels.ordinal_model.OrderedModel.html
model_pl = OrderedModel(endog=y_pl,
                       exog=X_pl_transformed,
                       distr="logit",
                        )
res_log = model_pl.fit(method='bfgs', disp=False)

In [55]:
res_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                ppltrst   Log-Likelihood:                -3035.7
Model:                   OrderedModel   AIC:                             6125.
Method:            Maximum Likelihood   BIC:                             6268.
Date:                Sat, 06 Jan 2024                                         
Time:                        00:21:46                                         
No. Observations:                1465                                         
Df Residuals:                    1438                                         
Df Model:                          17                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
gndr_Male                                    0.0993      0.095      1.048      0.295      -0.086       0.285
domicil_Country village                      0.0238      0.132      0.180      0.857      -0.236       0.283
domicil_Farm or home in countryside         -0.2346      0.262     -0.894      0.371      -0.749       0.280
domicil_Suburbs or outskirts of big city     0.1475      0.227      0.650      0.515      -0.297       0.592
domicil_Town or small city                   0.0124      0.124      0.100      0.920      -0.231       0.256
hinctnta_D - 9th decile                      0.3707      0.215      1.724      0.085      -0.051       0.792
hinctnta_F - 5th decile                     -0.0717      0.213     -0.336      0.737      -0.490       0.346
hinctnta_H - 10th decile                     0.4566      0.214      2.135      0.033       0.037       0.876
hinctnta_J - 1st decile                      0.1351      0.231      0.585      0.559      -0.318       0.588
hinctnta_K - 7th decile                      0.0903      0.220      0.411      0.681      -0.340       0.521
hinctnta_M - 4th decile                      0.0772      0.212      0.365      0.715      -0.337       0.492
hinctnta_P - 8th decile                      0.1137      0.212      0.536      0.592      -0.302       0.530
hinctnta_R - 2nd decile                      0.3763      0.208      1.809      0.070      -0.031       0.784
hinctnta_S - 6th decile                      0.3199      0.216      1.480      0.139      -0.104       0.743
uemp3m_Yes                                  -0.1125      0.100     -1.127      0.260      -0.308       0.083
agea                                        -0.0014      0.003     -0.524      0.600      -0.007       0.004
eduyrs                                       0.0809      0.015      5.267      0.000       0.051       0.111
You can't be too careful/1                   0.4741      0.333      1.423      0.155      -0.179       1.127
1/2                                         -1.0492      0.091    -11.503      0.000      -1.228      -0.870
2/3                                         -0.6913      0.072     -9.642      0.000      -0.832      -0.551
3/4                                         -0.7544      0.074    -10.190      0.000      -0.900      -0.609
4/5                                         -1.2072      0.098    -12.260      0.000      -1.400      -1.014
5/6                                         -0.3051      0.068     -4.460      0.000      -0.439      -0.171
6/7                                         -1.0949      0.123     -8.934      0.000      -1.335      -0.855
7/8                                         -0.6517      0.114     -5.709      0.000      -0.875      -0.428
8/9                                         -0.2376      0.122     -1.946      0.052      -0.477       0.002
9

### Model for Sweden

In [21]:
data_se = data[data.cntry == "SE"].drop(columns=["cntry"])

In [ ]:
profile = ProfileReport(data_se, title="Profiling Report for Sweden")
profile.to_file("../data/EDA_SE.html")

In [ ]:
model_se = OrderedModel(endog=None,
                       exog=None,
                       distr="logit")
res_log = model_se.fit(method='bfgs', disp=False)
res_log.summary()